# KG Generator Tool

## Let's get started by importing relevant libraries

In [ ]:
#pip install -r requirements.txt

In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm.auto import tqdm

import pybel
from pybel.io.jupyter import to_jupyter
from utils import *
from kg_gen_2 import *

from IPython.display import Markdown, display

## Please execute following cells

In [ ]:
kg = createKG()

In [ ]:
kg.summarize

In [ ]:
to_jupyter(kg)

In [ ]:
KG_namespace_plot(kg)

In [ ]:
#For this example we are going to use the python default http library
import requests

#Build a dict with the following format, change the value of the two keys your DisGeNET account credentials, if you don't have an account you can create one here https://www.disgenet.org/signup/ 
auth_params = {"email":"change@this.email","password":"changethis"}

api_host = "https://www.disgenet.org/api"

api_key = None
s = requests.Session()
try:
    r = s.post(api_host+'/auth/', data=auth_params)
    if(r.status_code == 200):
        #Lets store the api key in a new variable and use it again in new requests
        json_response = r.json()
        api_key = json_response.get("token")
        print(api_key + "This is your user API key.") #Comment this line if you don't want your API key to show up in the terminal
    else:
        print(r.status_code)
        print(r.text)
except requests.exceptions.RequestException as req_ex:
    print(req_ex)
    print("Something went wrong with the request.")

if api_key:
    #Add the api key to the requests headers of the requests Session object in order to use the restricted endpoints.
    s.headers.update({"Authorization": "Bearer %s" % api_key}) 
    #Lets get all the diseases associated to a gene eg. APP (EntrezID 351) and restricted by a source.
    gda_response = s.get(api_host+'/gda/gene/351', params={'source':'UNIPROT'})
    print(gda_response.json())

if s:
    s.close()

In [ ]:
#For this example we are going to use the python default http library
import requests

#Build a dict with the following format, change the value of the two keys your DisGeNET account credentials, if you don't have an account you can create one here https://www.disgenet.org/signup/ 
auth_params = {"email":"reagonkarki@gmail.com","password":"Bhunti.87"}

api_host = "https://www.disgenet.org/api"

api_key = 'e25cb13382cb9b016247822c49f325f75991e607'
s = requests.Session()
try:
    r = s.post(api_host+'/auth/', data=auth_params)
    if(r.status_code == 200):
        #Lets store the api key in a new variable and use it again in new requests
        json_response = r.json()
        api_key = json_response.get("token")
        print(api_key + "This is your user API key.") #Comment this line if you don't want your API key to show up in the terminal
    else:
        print(r.status_code)
        print(r.text)
except requests.exceptions.RequestException as req_ex:
    print(req_ex)
    print("Something went wrong with the request.")

if api_key:
    #Add the api key to the requests headers of the requests Session object in order to use the restricted endpoints.
    s.headers.update({"Authorization": "Bearer %s" % api_key}) 
    #Lets get all the diseases associated to a gene eg. APP (EntrezID 351) and restricted by a source.
    
    #https://www.disgenet.org/api/vda/disease/D000544
        
    gda_response = s.get(api_host+'/vda/disease/mondo/0004975?type=disease&format=json')
    #gda_response = s.get(api_host+'/gda/gene/351', params={'source':'UNIPROT'})
    print(gda_response.json())

if s:
    s.close()

In [ ]:
def GetDiseaseSNPs(disease_id): 
    
    import requests
    
    doid = disease_id.split("_")
    print(doid)
    
    #For this example we are going to use the python default http library
    

    #Build a dict with the following format, change the value of the two keys your DisGeNET account credentials, if you don't have an account you can create one here https://www.disgenet.org/signup/ 
    auth_params = {"email":"reagonkarki@gmail.com","password":"Bhunti.87"}

    api_host = "https://www.disgenet.org/api"

    api_key = 'e25cb13382cb9b016247822c49f325f75991e607'
    s = requests.Session()

    if api_key:
        #Add the api key to the requests headers of the requests Session object in order to use the restricted endpoints.
        s.headers.update({"Authorization": "Bearer %s" % api_key}) 
        #Lets get all the diseases associated to a gene eg. APP (EntrezID 351) and restricted by a source.

        #https://www.disgenet.org/api/vda/disease/D000544
    
        gda_response = s.get(api_host+'/vda/disease/'+str(doid[0]).lower()+ "/" +str(doid[1]) +'?format=json')
        #gda_response = s.get(api_host+'/gda/gene/351', params={'source':'UNIPROT'})

        
        gda_response = gda_response.json()
        gda_response = pd.DataFrame(gda_response)
        return(gda_response)

    if s:
        s.close()

In [ ]:
test = GetDiseaseSNPs('MONDO_0004975')

In [ ]:
test = GetDiseaseSNPs('EFO_0000765')

In [ ]:
test

In [ ]:
x ='OR5V1;OR12D3'
#x = 'OR5V1'
x = x.split(';')
len(x)

In [ ]:
def snp2gene_rel(snp_df,graph): 
    
    #convert col to datatype == str to remove rows that have 'None' in gene_symbol
    snp_df[['gene_symbol']] =  snp_df[['gene_symbol']].astype(str)
    snp_df = snp_df.loc[snp_df['gene_symbol'] != "None"]
    snp_df = snp_df.reset_index(drop=True)
    
    print(len(snp_df))
    
    for i in tqdm(range(len(snp_df)),desc='adding SNPs'):
        genes = snp_df['gene_symbol'][i].split(';')
        
        for j in range(len(genes)):
        
            graph.add_association(
                Gene(namespace="dbSNP",name=snp_df['variantid'][i]),
                Protein(namespace = "HGNC", name = genes[j]),
                citation = "DisGeNet",
                evidence = "SNPs for queried disease"
            )
    
    return(graph)

In [ ]:
from pybel.dsl import Gene
kg = pybel.BELGraph(name='test', version="0.0.1")

In [ ]:
kg = snp2gene_rel(test,kg)

In [ ]:
kg = dbSNP_annotation(kg)

In [ ]:
kg.summarize

In [ ]:
to_jupyter(kg)

In [ ]:
#list entities of interest separated by a space
entityList = 'rs763059810 rs2395029 rs2284178'.split(' ')

#prtn_attr = nx.get_node_attributes(kg,'UniProt')
chem_attr = nx.get_node_attributes(kg,'source')
#bionote_attr = nx.get_node_attributes(mpox_graph,'3Dbio')

for node in kg.nodes():
    unique_node = []
    
    if node.name in entityList:
        if node not in unique_node:
            
            unique_node.append(node)
            print(node.name)
       
            if isinstance(node,pybel.dsl.Gene):
                print(chem_attr[node])
                
#             if isinstance(node,pybel.dsl.Protein):
#                 print(prtn_attr[node])
                
#             if isinstance(node,pybel.dsl.Protein):
#                 print(bionote_attr[node])
            

In [ ]:
chem_attr